# Synthetic Line

One of the easiest ways to build a model predicting events is to use some combination of the markets, or sportsbooks. In data science, this is akin to creating a "stacked" model where the outputs of multiple models are combined in a linear or non-linear model to come up with an optimized scheme. Linear models can be thought of as simply weighting each market. For example, using historical data, we may find that Draftkings head-to-head round matchup odds are the sharpest followed by Pinnacle, and then Bookmaker. A model can provide the optimal weights that each sportsbook would get. Using this example, it may be 50% Draftkings, 30% Pinnacle, and 20% Bookmaker.

The goal of this analysis is to come up with an optimized synthetic line for PGA Tour head-to-head **round** matchups.

## Package Import

In [25]:
import pandas as pd
import numpy as np
from google.cloud import bigquery

import matplotlib.pyplot as plt
plt.style.use('ggplot')

import seaborn as sns

pd.set_option('max_columns', 100)

client = bigquery.Client()

## Data Import

Using DataGolf's API, I have historical round matchups across various sportsbooks. In addition to the historical data, I've created a BigQuery table that includes the sportsbooks that DataGolf currently supports. After all, we need to be able to get new data when making future predictions with a model.

### Active Sportsbooks

In [7]:
sportsbook_query = client.query("SELECT * FROM `golf-predict.entities.sportsbooks` WHERE active = 1")

sportsbooks = sportsbook_query.result().to_dataframe()

In [21]:
(sportsbooks
 .query("odds_type == 'matchups'")
 .sort_values(['start_date'])
 .reset_index(drop=True)
)

,sportsbook,odds_type,start_date,end_date,active
0,bet365,matchups,2019-01-20,None,1
1,bovada,matchups,2019-01-20,None,1
2,pinnacle,matchups,2019-03-26,None,1
3,draftkings,matchups,2019-05-29,None,1
4,betcris,matchups,2019-10-09,None,1
5,betonline,matchups,2020-02-11,None,1
6,fanduel,matchups,2020-10-06,None,1
7,betmgm,matchups,2021-01-15,None,1
8,pointsbet,matchups,2021-05-05,None,1
9,circa,matchups,2022-01-04,None,1


We have 11 active sportsbooks that have historical matchup data. Some (Circa, Unibet) are more recent than others, but we will explore using only 2022 data to predict the matchups that have occurred throughout 2023 thus far.

In [27]:
active_books = sportsbooks.query("odds_type == 'matchups'")['sportsbook']
active_books

0         bet365
1        betcris
2         betmgm
3      betonline
4         bovada
5          circa
6     draftkings
7        fanduel
8       pinnacle
9      pointsbet
10        unibet
Name: sportsbook, dtype: object

### Historical Matchups

In [22]:
matchups_query = client.query("SELECT * FROM `golf-predict.historical_betting_odds.matchups`")

matchups = matchups_query.result().to_dataframe()

In [23]:
matchups.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311682 entries, 0 to 311681
Data columns (total 28 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   bet_type         311578 non-null  object             
 1   close_time       311578 non-null  datetime64[ns, UTC]
 2   open_time        311578 non-null  datetime64[ns, UTC]
 3   p1_close         311578 non-null  float64            
 4   p1_dg_id         311578 non-null  float64            
 5   p1_open          311578 non-null  float64            
 6   p1_outcome       311578 non-null  float64            
 7   p1_outcome_text  311578 non-null  object             
 8   p1_player_name   311578 non-null  object             
 9   p2_close         311578 non-null  float64            
 10  p2_dg_id         311578 non-null  float64            
 11  p2_open          311578 non-null  float64            
 12  p2_outcome       311578 non-null  float64            
 13 

I would typically do a lot of filtering and such within the SQL query, but simply importing all data to do a lot more work with Python

------
shortcuts: 
- j/k up and down
- a add above, b add below
- m markdown, y code
------

In [26]:
matchups.columns

Index(['bet_type', 'close_time', 'open_time', 'p1_close', 'p1_dg_id',
       'p1_open', 'p1_outcome', 'p1_outcome_text', 'p1_player_name',
       'p2_close', 'p2_dg_id', 'p2_open', 'p2_outcome', 'p2_outcome_text',
       'p2_player_name', 'p3_close', 'p3_dg_id', 'p3_open', 'p3_outcome',
       'p3_outcome_text', 'p3_player_name', 'tie_rule', 'book',
       'event_completed', 'event_name', 'season', 'year', 'event_id'],
      dtype='object')

In [28]:
keep_cols = [
    'bet_type', 'close_time', 'open_time', 
    'p1_close', 'p1_dg_id', 'p1_open', 'p1_outcome', 'p1_outcome_text', 'p1_player_name',
    'p2_close', 'p2_dg_id', 'p2_open', 'p2_outcome', 'p2_outcome_text', 'p2_player_name', 
    'tie_rule', 'book', 'event_completed', 'event_name', 'season', 'year', 'event_id'
]

In [99]:
matchups.head()

,bet_type,close_time,open_time,p1_close,p1_dg_id,p1_open,p1_outcome,p1_outcome_text,p1_player_name,p2_close,p2_dg_id,p2_open,p2_outcome,p2_outcome_text,p2_player_name,p3_close,p3_dg_id,p3_open,p3_outcome,p3_outcome_text,p3_player_name,tie_rule,book,event_completed,event_name,season,year,event_id
0,None,NaT,NaT,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,5dimes,2023-05-28,Charles Schwab Challenge,2023,2023,21
1,None,NaT,NaT,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,sportsbook,2023-05-28,Charles Schwab Challenge,2023,2023,21
2,None,NaT,NaT,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,williamhill,2023-05-28,Charles Schwab Challenge,2023,2023,21
3,None,NaT,NaT,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,5dimes,2023-05-21,PGA Championship,2023,2023,33
4,None,NaT,NaT,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,NaN,NaN,NaN,NaN,None,None,None,sportsbook,2023-05-21,PGA Championship,2023,2023,33


In [101]:
matchups = matchups[keep_cols].copy()

Some factors we want to consider as we filter our base population for modeling. 
1. Ties are void (ie two-way markets)
2. Round matchups only
3. From an active sportsbook

In [109]:
print(matchups.tie_rule.unique())
print(matchups.bet_type.unique())

[None 'Bets are void in event of tie' 'Separate bet offered for tie'
 'Ties settled by dead-heat rules']
[None 'R1 Match-Up' 'R2 Match-Up' 'R3 Match-Up' 'R4 Match-Up'
 '72-hole Match' 'R1 3-Ball' 'R2 3-Ball' 'R3 3-Ball' 'R4 3-Ball']


In [110]:
# creating round list for filtering
round_list = list(map(lambda x: 'R' + x, list(map(str, range(1,5)))))

# filter criteria
tie_rule_filt = matchups.tie_rule == 'Bets are void in event of tie'
sportsbk_filt = matchups.book.str.contains('|'.join(active_books))
bet_type_filt = matchups.bet_type.str.contains('|'.join(round_list))

df = matchups.loc[tie_rule_filt & sportsbk_filt & bet_type_filt].reset_index(drop=True)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147422 entries, 0 to 147421
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype              
---  ------           --------------   -----              
 0   bet_type         147422 non-null  object             
 1   close_time       147422 non-null  datetime64[ns, UTC]
 2   open_time        147422 non-null  datetime64[ns, UTC]
 3   p1_close         147422 non-null  float64            
 4   p1_dg_id         147422 non-null  float64            
 5   p1_open          147422 non-null  float64            
 6   p1_outcome       147422 non-null  float64            
 7   p1_outcome_text  147422 non-null  object             
 8   p1_player_name   147422 non-null  object             
 9   p2_close         147422 non-null  float64            
 10  p2_dg_id         147422 non-null  float64            
 11  p2_open          147422 non-null  float64            
 12  p2_outcome       147422 non-null  float64            
 13 

In [142]:
# change a few types
df.p1_dg_id = df.p1_dg_id.astype(int)
df.p2_dg_id = df.p2_dg_id.astype(int)

In [115]:
df[['book','year']].value_counts()

book        year
pinnacle    2022    10436
betcris     2022     7756
            2021     7678
bovada      2021     7542
betonline   2021     6632
bovada      2022     6563
draftkings  2021     6215
betonline   2022     6201
unibet      2022     5667
pinnacle    2021     5631
bovada      2020     5168
betcris     2020     5108
draftkings  2020     4620
bet365      2021     4334
betonline   2020     4203
draftkings  2022     4202
pinnacle    2020     4199
bet365      2022     3990
bovada      2019     3934
pinnacle    2019     3575
bovada      2023     3507
pinnacle    2023     3334
bet365      2020     3222
betcris     2023     2923
bet365      2019     2903
betonline   2023     2586
unibet      2023     2287
betmgm      2022     2247
circa       2022     1881
betmgm      2021     1862
draftkings  2023     1694
circa       2023     1628
bet365      2023     1452
betmgm      2023      973
betcris     2019      918
fanduel     2023      203
            2022      131
            2021     

In [145]:
# create a unique ID
df['matchup_id'] = (
    df[['year','season','event_id','p1_dg_id','p2_dg_id','bet_type']]
    .apply(lambda x: x.astype(str))
    .apply(lambda x: '-'.join(x), axis=1)
)

#df.loc[df.duplicated(keep=False)]

In [151]:
dup_matchups = df.loc[df[['matchup_id','book']].duplicated(keep=False)]

In [154]:
dup_matchups.book.value_counts()

bovada        588
draftkings     80
pinnacle        2
Name: book, dtype: int64

In [161]:
# removing these books based on industry knowledge
remove_book = ['bovada','pointsbet','unibet']

In [175]:
df = df[~df.book.isin(remove_book)].reset_index(drop=True).copy()

In [179]:
df = df[~df.duplicated(subset=['book','matchup_id'])].reset_index(drop=True).copy()

In [211]:
recent_df = df.query("year.isin([2022,2023])")

(pd.crosstab(recent_df['matchup_id'], recent_df['book'])
 .reset_index()
 .query("betmgm == 1")
 .drop(['matchup_id'], axis=1)
 .apply(lambda x: sum(x))
)

book
bet365         154
betcris        977
betmgm        3220
betonline      979
circa          186
draftkings    2006
fanduel        100
pinnacle      1507
dtype: int64

In [198]:
matchup_crosstab = pd.crosstab(df['matchup_id'], df['book']).reset_index()

In [205]:
# most common offerings to draftkings
(matchup_crosstab.loc[matchup_crosstab.draftkings == 1]
 .drop(['matchup_id'], axis = 1)
 .apply(lambda x: sum(x))
)

book
bet365         1505
betcris        3191
betmgm         3459
betonline      5643
circa           321
draftkings    16691
fanduel          87
pinnacle       8077
dtype: int64

# TO DO

Create a function that will develop the optimal synthetic lines for 2022, evaluate on 2023 matchups. Do this for Illinois books (BetMGM, Draftkings).